In [99]:
# AlphaGo notebook
import sgf
import numpy as np
from openTSNE import TSNE
from openTSNE.callbacks import ErrorLogger
from sgfmill import boards
from sgfmill import ascii_boards
from sgfmill import sgf_moves
from sgfmill import sgf as sgf2
import os

delete_duplicates = True



class GoBoard:
    def __init__(self):
        # create board as dictionary
        self.board = {}
        self.sgf = boards.Board(19)

        
    def encode(self, value):
        if value == 'n':
            return [ 1, 0, 0 ]
        if value == 'b':
            return [ 0, 1, 0 ]
        if value == 'w':
            return [ 0, 0, 1 ]
        
        
    def toFeatureVector(self):
        vector = []
        for x in range(0, 19):
            for y in range(0, 19):
                key = chr(ord('a') + x) + chr(ord('a') + y)
                col = ord(key[0]) - ord('a')
                row = 18 - (ord(key[1]) - ord('a'))
                if (self.sgf.get(row, col) != None):
                    vector = vector + self.encode(self.sgf.get(row, col))
                else:
                    vector = vector + self.encode('n')
                
                
                #if key in self.board:
                 #   vector = vector + self.encode(self.board[key])
                #else:
                 #   vector = vector + self.encode('n')
        return vector
    
    def performMove(self, position, player):
        self.board[position] = player
        col = ord(position[0]) - ord('a')
        row = 18 - (ord(position[1]) - ord('a'))
        
        
        self.sgf.play(row, col, player.lower())
        
    def play(self, move):

        if (move[1] != None):
            self.sgf.play(move[1][0], move[1][1], move[0])
        
    def toVector(self):
        vector = []
        for x in range(0, 19):
            for y in range(0, 19):
                key = chr(ord('a') + x) + chr(ord('a') + y)
                col = ord(key[0]) - ord('a')
                row = 18 - (ord(key[1]) - ord('a'))
                if (self.sgf.get(row, col) != None):
                    vector = vector + [self.sgf.get(row, col)]
                else:
                    vector = vector + ['n']
                
                #if key in self.board:
                #    vector = vector + [self.board[key]]
                #else:
                #    vector = vector + ['n']
        return vector

In [100]:
encoded = []
games = []

class GG:
    def __init__(self, nodes, algo, line):
        self.nodes = nodes
        self.algo = algo
        self.line = line

algos = [ 'agzero_vs_agmaster', 'agzero_vs_agzero_40', 'tygemfox' ]

for algo in algos:
    print(f"alphago/{algo}")
    files = [f for f in os.listdir(f"alphago/{algo}")]
    print(files)
    for filename in files:
        print(filename)
        with open(f"alphago/{algo}/{filename}") as f:
            content = f.read()
            collection = sgf.parse(content)
            sgf2game = sgf2.Sgf_game.from_string(content)
            movelist = sgf_moves.get_setup_and_moves(sgf2game)

            g = []
            board = GoBoard()
            for node in movelist[1]:
                if (node[1] != None):
                    board.play(node)
                    encoded.append(board.toFeatureVector())
                    g.append(board.toVector())

            sgfboard = boards.Board(19)

            line = filename
            if line.startswith('Game'):
                line = f"{algo} {line}"
            games.append(GG(g, algo, line))

            
encoded, indices, counts = np.unique(encoded, axis=0, return_inverse=True, return_counts=True)

alphago/agzero_vs_agmaster
['Game_020.sgf', 'Game_013.sgf', 'Game_015.sgf', 'Game_003.sgf', 'Game_018.sgf', 'Game_011.sgf', 'Game_006.sgf', 'Game_004.sgf', 'Game_001.sgf', 'Game_002.sgf', 'Game_014.sgf', 'Game_019.sgf', 'Game_005.sgf', 'Game_008.sgf', 'Game_016.sgf', 'Game_007.sgf', 'Game_012.sgf', 'Game_009.sgf', 'Game_017.sgf', 'Game_010.sgf']
Game_020.sgf
Game_013.sgf
Game_015.sgf
Game_003.sgf
Game_018.sgf
Game_011.sgf
Game_006.sgf
Game_004.sgf
Game_001.sgf
Game_002.sgf
Game_014.sgf
Game_019.sgf
Game_005.sgf
Game_008.sgf
Game_016.sgf
Game_007.sgf
Game_012.sgf
Game_009.sgf
Game_017.sgf
Game_010.sgf
alphago/agzero_vs_agzero_40
['Game_020.sgf', 'Game_013.sgf', 'Game_015.sgf', 'Game_003.sgf', 'Game_018.sgf', 'Game_011.sgf', 'Game_006.sgf', 'Game_004.sgf', 'Game_001.sgf', 'Game_002.sgf', 'Game_014.sgf', 'Game_019.sgf', 'Game_005.sgf', 'Game_008.sgf', 'Game_016.sgf', 'Game_007.sgf', 'Game_012.sgf', 'Game_009.sgf', 'Game_017.sgf', 'Game_010.sgf']
Game_020.sgf
Game_013.sgf
Game_015.sgf
Game

In [101]:
tsne = TSNE(
    perplexity=100,
    n_jobs=6,
    metric='euclidean',
    random_state=42
)

In [102]:
%time embedding = tsne.fit(np.array(encoded))

embedding = embedding[indices]



CPU times: user 5min 21s, sys: 8min 14s, total: 13min 36s
Wall time: 1min 26s


In [103]:

embedding_split = np.array_split(embedding, np.add.accumulate([len(l.nodes) for l in games]))[:-1]
print(embedding_split)

[TSNEEmbedding([[ -4.07147156,   0.77118466],
               [ -3.91479234,   0.47246566],
               [ -3.62179536,   0.18884012],
               [ -3.92220608,  -1.08733819],
               [ -2.92924742,  -3.6458368 ],
               [ -2.71490257,  -3.79735162],
               [ -2.37838765,  -4.01286441],
               [ -2.08917368,  -4.18084337],
               [ -1.86917219,  -4.30022688],
               [ -1.72690176,  -4.35607499],
               [ -1.63457075,  -4.37573252],
               [ -1.55655507,  -4.39916082],
               [ -1.46773347,  -4.40477978],
               [ -1.40277732,  -4.4098177 ],
               [ -1.36174387,  -4.40739938],
               [ -1.32035083,  -4.40719809],
               [ -1.28752581,  -4.40973203],
               [ -1.25810866,  -4.40861646],
               [ -1.21429081,  -4.40475228],
               [ -1.17858037,  -4.40192403],
               [ -1.15541313,  -4.40325264],
               [ -1.11821546,  -4.40212745],
         

In [105]:
csv = open("gotest.csv", "w")
csv.write("x,y,line,algo")
for x in range(0, 19):
    for y in range(0, 19):
        key = chr(ord('a') + x) + chr(ord('a') + y)
        csv.write(',')
        csv.write(key)
csv.write("\n")
idx = 0

print(len(embedding))
print(len(games))
print(len(games[0].nodes))


for i, game in enumerate(games):
    for state in game.nodes:
        
        csv.write(str(embedding[idx][0]))
        csv.write(',')
        csv.write(str(embedding[idx][1]))
        csv.write(',')
        csv.write(game.line)
        csv.write(',')
        csv.write(game.algo)
        idx = idx + 1
        for fa in state:
            csv.write(',')
            csv.write(str(fa))
        csv.write('\n')
        
csv.close()

21783
100
265
